In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from glob import glob
import os

In [2]:
spark = SparkSession.builder.getOrCreate()

24/11/17 07:58:30 WARN Utils: Your hostname, hyle-nitro resolves to a loopback address: 127.0.1.1; using 192.168.0.216 instead (on interface wlp0s20f3)
24/11/17 07:58:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/17 07:58:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# get all .txt file paths in NER 2016 dataset
# text_file_paths = sorted(glob("./data/NER2016-TestData-16-9-2016/**/*.txt", recursive=True))
text_file_paths = sorted(glob("./data/NER2016-TrainingData-3-3-2017-txt/**/*.txt", recursive=True))
origin = "train"

In [4]:
tag_names = ["B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "O"]

In [5]:
# read file and extract tags
def extract_and_reformat(text_file_paths):
    for file_counter, filepath in enumerate(text_file_paths):
        with open(filepath, "r", encoding="utf-8") as f:
            content = f.readline()
            output_file = open(f"./data/formatted_data/2016/{origin}/{str(file_counter).zfill(5)}.txt", "w", encoding="utf-8")

            line_counter = 0

            while content:
                if line_counter <= 3:
                    line_counter += 1
                    content = f.readline()
                    continue
                # get tags
                tags = content.split("\t")

                # if tag is not a word
                if tags[0] == '<s>' or tags[0] == "</s>":
                    tags = []
                else:
                    tags[-1] = tags[-1].strip()

                if tags and len(tags) >= 4:
                    words = tags[0]
                    ner_tag = tags[3]
                    output_file.write(f"{words} {ner_tag}\n")

                content = f.readline()
            
            # prepare for next increment
            output_file.close()

extract_and_reformat(text_file_paths)

In [6]:
df = spark.read.text(text_file_paths, lineSep="\n")

In [7]:
rdd = df.rdd.map(lambda x: Row(length=len(x["value"].split("\t"))))

In [8]:
schema = StructType([
     StructField("length", IntegerType(), True)
])
df2 = rdd.toDF(schema=schema)

In [9]:
# df2 = df2.collect()

# select function returns another dataframe that can be used to show
# df_word = df2.select("word")

df2.show()

+------+
|length|
+------+
|     1|
|     1|
|     1|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
|     5|
+------+
only showing top 20 rows



24/11/17 16:28:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 18778258 ms exceeds timeout 120000 ms
24/11/17 16:28:22 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/17 16:28:25 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint